<a href="https://colab.research.google.com/github/ParkEunHyeok/AI_Study/blob/main/NLP/seq2seq_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 50.0 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

from konlpy.tag import Okt

In [ ]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

In [ ]:
# 구글 드라이브 연결
import os
from google.colab import drive
drive.mount('/content/gdrive/')
path = "gdrive/My Drive/Colab Notebooks/seq2seq"

Mounted at /content/gdrive/


In [ ]:
wellness = pd.read_csv("gdrive/My Drive/Colab Notebooks/HelloNewWorld/wellnessData.csv")

In [ ]:
# 결측 row 제거하는 방법, 5232개
wellness.dropna(how='any')
wellness[:10]

,구분,유저,챗봇,라벨
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,0.0
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,0.0
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",0.0
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,"그럴 때 마음을 편하게 가지고, 기분 좋은 생각을 한번 해보는건 어떨까요.",0.0
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,저도 그 기분 이해해요. 많이 힘드시죠?,0.0
5,감정/감정조절이상,작년 가을부터 감정조절이 잘 안 되는 거 같아.,"그럴 때 마음을 편하게 가지고, 기분 좋은 생각을 한번 해보는건 어떨까요.",0.0
6,감정/감정조절이상,예전에 비해서 인내심이 너무 짧아진 거 같아.,요즈음 힘든 일이 많으신가봐요.,0.0
7,감정/감정조절이상,더 이상 혼자서는 감정조절을 못하겠어.,그럴 때는 어디에 털어놓아 보는건 어떨까요.,0.0
8,감정/감정조절이상,점점 나 자신을 컨트롤하지 못하는 기분이야.,그럴 때는 어디에 털어놓아 보는건 어떨까요.,0.0
9,감정/감정조절이상,나도 이러기 싫은데 내 마음대로 안돼.,저도 그 기분 이해해요. 많이 힘드시죠?,0.0


In [ ]:
question = []
answer = []

for sentence in wellness['유저']:
    question.append(sentence)

for sentence in wellness['챗봇']:
    answer.append(sentence)

list(zip(question, answer))[:5]

[('제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.',
  '감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.'),
 ('더 이상 내 감정을 내가 컨트롤 못 하겠어.', '저도 그 기분 이해해요. 많이 힘드시죠?'),
 ('하루종일 오르락내리락 롤러코스터 타는 기분이에요.', '그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.'),
 ('꼭 롤러코스터 타는 것 같아요.', '그럴 때 마음을 편하게 가지고, 기분 좋은 생각을 한번 해보는건 어떨까요.'),
 ('롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.', '저도 그 기분 이해해요. 많이 힘드시죠?')]

In [ ]:
question = question[:3500]
answer = answer[:3500]

In [ ]:
# 형태소분석 함수
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER, "", sentence)
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

In [ ]:
# 형태소분석 수행
question = pos_tag(question)
answer = pos_tag(answer)

# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()

Q : 제 감정 이 이상해진 것 같아요 남편 만 보면 화가 치밀어 오르고 감정 조절 이 안되요
A : 감정 이 조절 이 안 될 때 만큼 힘들 때 는 없는 거 같아요

Q : 더 이상 내 감정 을 내 가 컨트롤 못 하겠어
A : 저 도 그 기분 이해해 요 많이 힘드시죠

Q : 하루 종일 오르락내리락 롤러코스터 타는 기분 이에요
A : 그럴 때 는 밥 은 잘 먹었는지 잠 은 잘 잤는지 체크 해보는 것 도 좋아요

Q : 꼭 롤러코스터 타는 것 같아요
A : 그럴 때 마음 을 편하게 가지 고 기분 좋은 생각 을 한번 해보는건 어떨까 요

Q : 롤러코스터 타는 것 처럼 기분 이 왔다 갔다 해 요
A : 저 도 그 기분 이해해 요 많이 힘드시죠

Q : 작년 가을 부터 감정 조절 이 잘 안 되는 거 같아
A : 그럴 때 마음 을 편하게 가지 고 기분 좋은 생각 을 한번 해보는건 어떨까 요

Q : 예전 에 비 해서 인내심 이 너무 짧아진 거 같아
A : 요즈음 힘든 일이 많으신 가봐요

Q : 더 이상 혼자 서는 감정 조절 을 못 하겠어
A : 그럴 때 는 어디 에 털어놓아 보는건 어떨까 요

Q : 점점 나 자신 을 컨트롤 하지 못 하는 기분 이야
A : 그럴 때 는 어디 에 털어놓아 보는건 어떨까 요

Q : 나도 이러기 싫은데 내 마음대로 안 돼
A : 저 도 그 기분 이해해 요 많이 힘드시죠



In [ ]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

In [ ]:
# 단어 개수
len(words)

7737

In [ ]:
words[:10]

['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '유일',
 '고요하고',
 '해였으면',
 '과분',
 '갔어요',
 '하던']

In [ ]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [ ]:
# 단어 -> 인덱스
# 문장을 인덱스로 변환하여 모델 입력으로 사용
dict(list(word_to_index.items())[:20])

{'<END>': 2,
 '<OOV>': 3,
 '<PADDING>': 0,
 '<START>': 1,
 '갔어요': 8,
 '고요하고': 5,
 '과분': 7,
 '낳으면서': 11,
 '문득': 12,
 '받았었거든': 19,
 '옮아오': 17,
 '원하는': 16,
 '유일': 4,
 '조심스러운': 15,
 '준비': 13,
 '출판사': 14,
 '하던': 9,
 '해본': 10,
 '해였으면': 6,
 '혈압': 18}

In [ ]:
# 인덱스 -> 단어
# 모델의 예측 결과인 인덱스를 문장으로 변환시 사용
dict(list(index_to_word.items())[:20])

{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '유일',
 5: '고요하고',
 6: '해였으면',
 7: '과분',
 8: '갔어요',
 9: '하던',
 10: '해본',
 11: '낳으면서',
 12: '문득',
 13: '준비',
 14: '출판사',
 15: '조심스러운',
 16: '원하는',
 17: '옮아오',
 18: '혈압',
 19: '받았었거든'}

In [ ]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

In [ ]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

# 첫 번째 인코더 입력 출력 (12시 땡)
x_encoder[0]

array([7573, 6850, 1164, 1508, 7724, 5274, 4757, 1495, 5843, 3121,  587,
       2877, 6850,  540, 1164, 4567,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [ ]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

# 첫 번째 디코더 입력 출력 (START 하루 가 또 가네요)
x_decoder[0]

array([   1, 6850, 1164,  540, 1164, 6305, 6278, 6672, 6472, 1721, 6672,
       5913, 4248,  544, 5274,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [ ]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 목표 출력 (하루 가 또 가네요 END)
y_decoder[0]

array([6850, 1164,  540, 1164, 6305, 6278, 6672, 6472, 1721, 6672, 5913,
       4248,  544, 5274,    2,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [ ]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))

# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [ ]:
#--------------------------------------------
# 훈련 모델 인코더 정의
#--------------------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]



#--------------------------------------------
# 훈련 모델 디코더 정의
#--------------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.2,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#--------------------------------------------
#  예측 모델 인코더 정의
#--------------------------------------------

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)

#--------------------------------------------
# 예측 모델 디코더 정의
#--------------------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

In [ ]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [ ]:
# 모델 분산 학습 할 때 체크포인트 저장
checkpoint_path = 'gdrive/My Drive/Colab Notebooks/HelloNewWorld/training_checkpoint-6.ckpt'
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True,
                             save_best_only=True, 
                             monitor='loss', 
                             verbose=1
                            )

In [ ]:
# 연속하여 학습시 체크포인트를 로드하여 이어서 학습합니다.
model.load_weights('gdrive/My Drive/Colab Notebooks/HelloNewWorld/training_checkpoint-6.ckpt')

In [ ]:
# 에폭 반복
for epoch in range(1):
    print('Total Epoch :', epoch + 1)

    # 훈련 시작
    history = model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=40,
                        batch_size=16,
                        verbose=1,
                        callbacks=[checkpoint])

Total Epoch : 1
Epoch 1/40
 14/219 [>.............................] - ETA: 54s - loss: 3.5265 - accuracy: 0.7064

KeyboardInterrupt: ignored

In [ ]:
model.save('gdrive/My Drive/Colab Notebooks/HelloNewWorld/model.h5')

In [ ]:
# 정확도와 손실 출력
# print('accuracy :', history.history['accuracy'][-1])
# print('loss :', history.history['loss'][-1])

# 문장 예측 테스트
# (3 박 4일 놀러 가고 싶다) -> (여행 은 언제나 좋죠)
# input_encoder = x_encoder[2].reshape(1, x_encoder[2].shape[0])
# input_decoder = x_decoder[2].reshape(1, x_decoder[2].shape[0])
# results = model.predict([input_encoder, input_decoder])

# 결과의 원핫인코딩 형식을 인덱스로 변환
# 1축을 기준으로 가장 높은 값의 위치를 구함
# indexs = np.argmax(results[0], 1) 

# 인덱스를 문장으로 변환
# sentence = convert_index_to_text(indexs, index_to_word)
# print(sentence)
# print()

In [ ]:
from keras.models import load_model
model = load_model('gdrive/My Drive/Colab Notebooks/HelloNewWorld/model.h5')

In [ ]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [ ]:
# 텍스트 생성
def generate_text(input_seq):
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
        
    return sentence

In [ ]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('나 너무 우울해')
input_seq

array([[3102, 7607, 1110,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])

In [ ]:

# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

'하겠다고 나갔어요 모임 배기 냥 받긴 공포 요구 싫고 울음 재밌을 '